In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [8]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        new_V = np.copy(V)
        for s in range(env.nS):
            v = 0
            for a in range(env.nA):
                action_prob = policy[s][a]
                immediate_reward = env.P[s][a][0][2]
                v_prim = 0
                for transition_tuple in env.P[s][a]:
                    next_state = transition_tuple[1]
                    transition_prob = transition_tuple[0]
                    v_prim += discount_factor * transition_prob * V[next_state]
                v += action_prob * (immediate_reward + v_prim)
            new_V[s] = v
        if (abs(V - new_V) < theta).all():
            print(V.reshape(env.shape))
            break
        V = new_V
    return np.array(V)

In [9]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

[[  0.         -13.99988715 -19.99983277 -21.99981286]
 [-13.99988715 -17.99985268 -19.99983389 -19.99983277]
 [-19.99983277 -19.99983389 -17.99985268 -13.99988715]
 [-21.99981286 -19.99983277 -13.99988715   0.        ]]


In [7]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)